In [ ]:
!pip install --q dataset
!pip install --q transformers
!pip install --q ohmeow-blurr --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.52 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 59.8 MB/s eta 0:00

# Importing Libraries and Dataset

In [ ]:
from fastai.text.all import *
from transformers.utils import logging as hf_logging

from datasets import load_dataset

import os, warnings, torch

from blurr.text.data.all import *
from blurr.text.modeling.all import *

warnings.simplefilter("ignore")
hf_logging.set_verbosity_error()

os.environ["TOKENIZERS_PARALLELISM"] = "false"

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")


## New Synthetic Dataset

In [ ]:
test_dict = {
                "ner_tags": [
                    [1, 2, 0, 0, 5, 6, 0, 0, 0, 0, 3, 4, 0],
                    [0, 0, 0, 3, 0, 0, 5, 6, 0, 0, 0, 0, 0],
                    [0, 0, 1, 2, 0, 0, 0, 0, 5, 6, 0, 0, 0],
                    [0, 1, 2, 0, 0, 0, 5, 6, 0, 0, 3, 4, 0, 0],
                    [0, 1, 2, 0, 0, 0, 3, 4, 0, 0, 5, 6, 0],
                    [0, 0, 0, 0, 5, 6, 6, 6, 6, 6, 6],
                    [0, 0, 0, 0, 5, 6, 6, 6, 0],
                    [0, 0, 0, 5, 6, 0, 5, 6, 6, 0],
                    [0, 0, 0, 0, 0, 5, 6, 6, 0],
                    [0, 0, 0, 5, 6, 6, 0, 5, 6, 0],
                    [0, 0, 0, 5, 6, 0, 5, 6, 0],
                    [0, 0, 0, 5, 6, 0, 5, 6, 0],
                    [0, 0, 0, 5, 6, 6, 6, 6, 0],
                    [1, 0, 0, 5, 6, 6, 0, 5, 6, 0, 5, 0]
                ],
                "tokens": [
                    ["John", "Doe", "lives", "on", "42nd", "Street", "and", "works", "for", "the", "United", "Nations", "."],
                    ["The", "meeting", "at", "Microsoft", "Headquarters", "on", "1st", "Avenue", "discussed", "new", "software", "updates", "."],
                    ["After", "leaving", "Elon", "Musk's", "presentation,", "I", "went", "to", "5th", "Avenue", "to", "shop", "."],
                    ["Does", "Jennifer", "Aniston", "still", "live", "on", "9th", "Avenue", "near", "the", "Warner", "Bros", "studio", "?"],
                    ["Is", "Tom", "Hanks", "currently", "filming", "at", "Paramount", "Pictures", "located", "on", "Melrose", "Avenue", "?"],
                    ["An", "active", "shooter", "in", "Flat", "No.", "565", "Corner", "Street", "New", "York"],
                    ["The", "house", "number", "is", "17", "on", "Main", "Street", "."],
                    ["I", "live", "in", "Apartment", "3B", "at", "123", "Elm", "Street", "."],
                    ["The", "office", "is", "located", "at", "30", "Rockefeller", "Plaza", "."],
                    ["I", "work", "at", "123", "Main", "Street", "in", "Apartment", "1C", "."],
                    ["She", "lives", "in", "Apartment", "2D", "on", "Fifth", "Avenue", "."],
                    ["I", "live", "at", "Apartment", "7C", "on", "Park", "Lane", "."],
                    ["The", "address", "is", "123", "Cherry", "Street", "Apartment", "2B", "."],
                    ["James", "lives", "at", "Flat", "No.", "101", "on", "Ocean", "Drive", "in", "Miami", "."]
                ]
            }

df = pd.DataFrame(test_dict)
df.head()

,ner_tags,tokens
0,"[1, 2, 0, 0, 5, 6, 0, 0, 0, 0, 3, 4, 0]","[John, Doe, lives, on, 42nd, Street, and, works, for, the, United, Nations, .]"
1,"[0, 0, 0, 3, 0, 0, 5, 6, 0, 0, 0, 0, 0]","[The, meeting, at, Microsoft, Headquarters, on, 1st, Avenue, discussed, new, software, updates, .]"
2,"[0, 0, 1, 2, 0, 0, 0, 0, 5, 6, 0, 0, 0]","[After, leaving, Elon, Musk's, presentation,, I, went, to, 5th, Avenue, to, shop, .]"
3,"[0, 1, 2, 0, 0, 0, 5, 6, 0, 0, 3, 4, 0, 0]","[Does, Jennifer, Aniston, still, live, on, 9th, Avenue, near, the, Warner, Bros, studio, ?]"
4,"[0, 1, 2, 0, 0, 0, 3, 4, 0, 0, 5, 6, 0]","[Is, Tom, Hanks, currently, filming, at, Paramount, Pictures, located, on, Melrose, Avenue, ?]"


## CONLL Dataset

In [ ]:
# raw_datasets = load_dataset("conll2003")
# conll2003_df = pd.DataFrame(raw_datasets["train"])
# conll2003_df.head()

In [ ]:
# additional_df = conll2003_df[["ner_tags", "tokens"]]
# additional_df.head()

In [ ]:
# print(df.shape)
# print(additional_df.shape)

# df_final = pd.concat([df, additional_df])
# df_final.shape

## Final Dataset

In [ ]:
LABELS = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
# df_final.head()

In [ ]:
# print('Data size: {}x{}'.format(df_final.shape[0], df_final.shape[1]))

# Getting the Model and Tokenizer

In [ ]:
from transformers import AutoModelForTokenClassification, AutoConfig

model_name = 'elastic/distilbert-base-uncased-finetuned-conll03-english'

model_cls = AutoModelForTokenClassification
config = AutoConfig.from_pretrained(model_name)

hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(
    model_name,
    model_cls=model_cls,
    config=config)

# Defining DataBlock and DataLoaders

In [ ]:
batch_tok_tfm = TokenClassBatchTokenizeTransform(
    hf_arch, hf_config, hf_tokenizer, hf_model)

blocks = (
    TextBlock(batch_tokenize_tfm=batch_tok_tfm, input_return_type=TokenClassTextInput),
    TokenCategoryBlock(vocab=LABELS))

dblock = DataBlock(
    blocks=blocks,
    get_x=ColReader("tokens"),
    get_y=ColReader("ner_tags"),
    splitter=RandomSplitter())

In [ ]:
dls = dblock.dataloaders(df, bs=2)

In [ ]:
dls.show_batch(dataloaders=dls)

,word / target label
0,"[('does', 'O'), ('jennifer', 'B-PER'), ('aniston', 'I-PER'), ('still', 'O'), ('live', 'O'), ('on', 'O'), ('9th', 'B-LOC'), ('avenue', 'I-LOC'), ('near', 'O'), ('the', 'O'), ('warner', 'B-ORG'), ('bros', 'I-ORG'), ('studio?', 'O')]"
1,"[('the', 'O'), ('meeting', 'O'), ('at', 'O'), ('microsoft', 'B-ORG'), ('headquarters', 'O'), ('on', 'O'), ('1st', 'B-LOC'), ('avenue', 'I-LOC'), ('discussed', 'O'), ('new', 'O'), ('software', 'O'), ('updates.', 'O')]"


# Defining the Learner

In [ ]:
model = BaseModelWrapper(hf_model)
fit_cbs = [TokenClassMetricsCallback()]

learn = Learner(
    dls,
    model,
    opt_func=partial(Adam),
    loss_func=PreCalculatedCrossEntropyLoss(),
    cbs=[BaseModelCallback],
    splitter=blurr_splitter)

In [ ]:
learn.fit_one_cycle(1, lr_max=3e-5, moms=(0.8, 0.7, 0.8), cbs=fit_cbs)

epoch,train_loss,valid_loss,accuracy,precision,recall,f1,time
0,0.757004,0.077227,0.956522,0.800000,0.666667,0.727273,00:04


In [ ]:
# learn.unfreeze()
# learn.fit_one_cycle(10)

In [ ]:
learn.show_results(learner=learn, trunc_at=250)

,token / target label / predicted label
0,"[('is', 'O', 'O'), ('tom', 'B-PER', 'B-PER'), ('hanks', 'I-PER', 'I-PER'), ('currently', 'O', 'O'), ('filming', 'O', 'O'), ('at', 'O', 'O'), ('paramount', 'B-ORG', 'B-ORG'), ('pictures', 'I-ORG', 'I-ORG'), ('located', 'O', 'O'), ('on', 'O', 'O'), ('melrose', 'B-LOC', 'B-LOC'), ('avenue?', 'I-LOC', 'I-LOC')]"
1,"[('i', 'O', 'O'), ('live', 'O', 'O'), ('in', 'O', 'O'), ('apartment', 'B-LOC', 'B-LOC'), ('3b', 'I-LOC', 'I-LOC'), ('at', 'O', 'O'), ('123', 'B-LOC', 'B-LOC'), ('elm', 'I-LOC', 'B-LOC'), ('street.', 'I-LOC', 'I-LOC')]"


# Prediction

In [ ]:
learn.blurr_predict_tokens(items=['An active shooter in Flat No. 565, Corner Street, New York'])

[[{'entity_group': 'LOC',
   'score': 0.9680487751960755,
   'word': 'Flat No. 565',
   'start': 21,
   'end': 33},
  {'entity_group': 'LOC',
   'score': 0.9895606935024261,
   'word': 'Corner Street',
   'start': 35,
   'end': 48},
  {'entity_group': 'LOC',
   'score': 0.996873140335083,
   'word': 'New York',
   'start': 50,
   'end': 58}]]

In [ ]:
# learn.save('12jul_model', with_opt=False)

Path('models/12jul_model.pth')

In [ ]:
export_fname = 'model'
learn.export(fname=f'{export_fname}.pkl')

In [ ]:
learn.export(fname=f'{export_fname}.pkl')
inf_learn = load_learner(fname=f'{export_fname}.pkl')

results = inf_learner.blurr_predict_tokens([txt, txt2])
for res in results:
    print(f'{res}\n')